> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Before submitting your project, it will be a good idea to go back through your report and remove these sections to make the presentation of your work as tidy as possible. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Soccer Database

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

### at the first, I export two csvs files from the data base by SQL

the first csv is contains all data about teams "teams.csv"
```
SELECT t.team_long_name, t.team_short_name,ta.*
FROM Team_Attributes  AS ta
LEFT JOIN Team AS t
ON ta.team_api_id = t.team_api_id 
	and ta.team_fifa_api_id = t.team_fifa_api_id;
```
the second csv is conatains all data about players "players.csv"
```
SELECT pa.player_api_id, pa.player_fifa_api_id, p.player_name,
		pa.date, pa.overall_rating, pa.penalties
FROM Player_Attributes  AS pa
LEFT JOIN Player AS p
ON pa.player_api_id = p.player_api_id 
    and pa.player_fifa_api_id = p.player_fifa_api_id;
```

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [70]:
df_teams = pd.read_csv('teams.csv')
df_teams.head()

,team_long_name,team_short_name,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,FC Aarau,AAR,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,FC Aarau,AAR,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,FC Aarau,AAR,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,Aberdeen,ABE,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,Aberdeen,ABE,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [71]:
#check missing data for dataframe df_teams and type of each columns 
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 27 columns):
team_long_name                    1458 non-null object
team_short_name                   1458 non-null object
id                                1458 non-null int64
team_fifa_api_id                  1458 non-null int64
team_api_id                       1458 non-null int64
date                              1458 non-null object
buildUpPlaySpeed                  1458 non-null int64
buildUpPlaySpeedClass             1458 non-null object
buildUpPlayDribbling              489 non-null float64
buildUpPlayDribblingClass         1458 non-null object
buildUpPlayPassing                1458 non-null int64
buildUpPlayPassingClass           1458 non-null object
buildUpPlayPositioningClass       1458 non-null object
chanceCreationPassing             1458 non-null int64
chanceCreationPassingClass        1458 non-null object
chanceCreationCrossing            1458 non-null int64
chanceCreationCrossi

In [72]:
# check duplicated rows for teams
sum(df_teams.duplicated())

0

In [73]:
# classes of Dribbling when build up Play Dribbling is null
df_teams[df_teams.buildUpPlayDribbling.isnull()]['buildUpPlayDribblingClass'].unique()

array(['Little'], dtype=object)

In [74]:
# classes of Dribbling
df_teams['buildUpPlayDribblingClass'].unique()

array(['Little', 'Normal', 'Lots'], dtype=object)

In [129]:
# define data frame for players
df_players = pd.read_csv('players.csv')
df_players.head()

,player_api_id,player_fifa_api_id,player_name,date,overall_rating,penalties
0,505942,218353,Aaron Appindangoye,2016-02-18 00:00:00,67.0,48.0
1,505942,218353,Aaron Appindangoye,2015-11-19 00:00:00,67.0,48.0
2,505942,218353,Aaron Appindangoye,2015-09-21 00:00:00,62.0,48.0
3,505942,218353,Aaron Appindangoye,2015-03-20 00:00:00,61.0,47.0
4,505942,218353,Aaron Appindangoye,2007-02-22 00:00:00,61.0,47.0


In [130]:
# check the type of columns and number of each row for df_players
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 6 columns):
player_api_id         183978 non-null int64
player_fifa_api_id    183978 non-null int64
player_name           183766 non-null object
date                  183978 non-null object
overall_rating        183142 non-null float64
penalties             183142 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 8.4+ MB


In [131]:
# check number of duplicated for df_players
sum(df_players.duplicated())

1

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (drop columns and clear missing values)

In [78]:
# remove nothing important columns to my Questions first for teams
# if column name contains name Class then rename it to True else let it default name
df_teams.rename(columns=lambda x: True if 'Class' in x else x , inplace = True)

In [86]:
df_teams.drop(columns = ['id',True], inplace = True)

In [88]:
df_teams.drop(columns = ['buildUpPlayDribbling'], inplace = True)

In [89]:
df_teams.head()

,team_long_name,team_short_name,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
0,FC Aarau,AAR,434,9930,2010-02-22 00:00:00,60,50,60,65,55,50,55,45
1,FC Aarau,AAR,434,9930,2014-09-19 00:00:00,52,56,54,63,64,47,44,54
2,FC Aarau,AAR,434,9930,2015-09-10 00:00:00,47,54,54,63,64,47,44,54
3,Aberdeen,ABE,77,8485,2010-02-22 00:00:00,70,70,70,70,70,60,70,70
4,Aberdeen,ABE,77,8485,2011-02-22 00:00:00,47,52,53,48,52,47,47,52


In [126]:
#convert type of date from string to timedate from df_teams
df_teams['date'] = pd.to_datetime(df_teams['date'])
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 13 columns):
team_long_name            1458 non-null object
team_short_name           1458 non-null object
team_fifa_api_id          1458 non-null int64
team_api_id               1458 non-null int64
date                      1458 non-null datetime64[ns]
buildUpPlaySpeed          1458 non-null int64
buildUpPlayPassing        1458 non-null int64
chanceCreationPassing     1458 non-null int64
chanceCreationCrossing    1458 non-null int64
chanceCreationShooting    1458 non-null int64
defencePressure           1458 non-null int64
defenceAggression         1458 non-null int64
defenceTeamWidth          1458 non-null int64
dtypes: datetime64[ns](1), int64(10), object(2)
memory usage: 148.2+ KB


In [132]:
#convert type of date from stirng to timedate from df_players
df_players['date'] = pd.to_datetime(df_players['date'])
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 6 columns):
player_api_id         183978 non-null int64
player_fifa_api_id    183978 non-null int64
player_name           183766 non-null object
date                  183978 non-null datetime64[ns]
overall_rating        183142 non-null float64
penalties             183142 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 8.4+ MB


In [140]:
#make copy for df_players and remove comlumn player_name from orginal
df_p_copy = df_players.copy()
df_players.drop(columns = 'player_name', inplace = True)
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 5 columns):
player_api_id         183978 non-null int64
player_fifa_api_id    183978 non-null int64
date                  183978 non-null datetime64[ns]
overall_rating        183142 non-null float64
penalties             183142 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 7.0 MB


In [142]:
#remove null values from df_players
df_players.dropna(inplace = True)
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183142 entries, 0 to 183977
Data columns (total 5 columns):
player_api_id         183142 non-null int64
player_fifa_api_id    183142 non-null int64
date                  183142 non-null datetime64[ns]
overall_rating        183142 non-null float64
penalties             183142 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 8.4 MB


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!